<a href="https://colab.research.google.com/github/MDfarazuddin99/CNN_Pruning/blob/master/Transfer_learning_pruning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import time
import os
import math
import numpy as np
import pandas as pd
# import GPyOpt
import keras
import random
from itertools import product, combinations
from collections import OrderedDict
from keras.preprocessing import image
from keras import layers, models, optimizers, callbacks, initializers
from keras.applications import DenseNet121, VGG16, ResNet50
from keras.callbacks import ReduceLROnPlateau
from keras.models import load_model
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras import optimizers
import numpy as np
from keras.layers.core import Lambda
from keras import backend as K
from keras import regularizers
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
# !pip install kerassurgeon
from kerassurgeon import identify 
from kerassurgeon.operations import delete_channels,delete_layer
from kerassurgeon import Surgeon


Using TensorFlow backend.


In [0]:

def normalize(X_train,X_test):
    #this function normalize inputs for zero mean and unit variance
    # it is used when training a model.
    # Input: training set and test set
    # Output: normalized training set and test set according to the trianing set statistics.
    mean = np.mean(X_train,axis=(0,1,2,3))
    std = np.std(X_train, axis=(0, 1, 2, 3))
    X_train = (X_train-mean)/(std+1e-7)
    X_test = (X_test-mean)/(std+1e-7)
    return X_train, X_test


def train(model, status):

    # uncomment the below code 

    #training parameters
    # DATA_FOLDER = "/home/shabbeer/Research/CalTech101"
    # # DATA_FOLDER = "CalTech101"
    # TRAIN_PATH = os.path.join(DATA_FOLDER, "training") # Path for training data
    # VALID_PATH = os.path.join(DATA_FOLDER, "validation") # Path for validation data

# delete the below code
# __________________________
    DATA_FOLDER = "/content/drive/My Drive/dog"
    # DATA_FOLDER = "CalTech101"
    TRAIN_PATH = os.path.join(DATA_FOLDER, "train") # Path for training data
    VALID_PATH = os.path.join(DATA_FOLDER, "val") # Path for validation data
# __________________________


    NUMBER_OF_CLASSES = len(os.listdir(TRAIN_PATH)) # Number of classes of the dataset
    EPOCHS = 20
    learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                                patience=5, 
                                                verbose=1, 
                                                factor=np.sqrt(0.1), 
                                                min_lr=0.0001)
    batch_size=8 # the mini-batch size to use for the dataset
    learning_rate = 0.1
    lr_decay = 1e-6
    lr_drop = 20
    datagen = image.ImageDataGenerator(preprocessing_function=keras.applications.vgg16.preprocess_input) # creating an instance of the data generator
    train_generator = datagen.flow_from_directory(TRAIN_PATH, target_size=(224, 224), batch_size=batch_size) # creating the generator for training data
    valid_generator = datagen.flow_from_directory(VALID_PATH, target_size=(224, 224), batch_size=batch_size) # creating the generator for validation data
    #Adagrad = optimizers.Adagrad(lr=0.1)
    def lr_scheduler(epoch):
        return learning_rate * (0.5 ** (epoch // lr_drop))
    reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)
    ADAM = optimizers.Adam(lr=0.1, beta_1=0.9, beta_2=0.999, amsgrad=False)
    model.compile(optimizer='ADAM', loss='categorical_crossentropy', metrics=['accuracy'])
    # model.summary()
    print(status)
    if status==True:  
        history = model.fit_generator(
        train_generator,
        validation_data=valid_generator, epochs=EPOCHS,
        steps_per_epoch=len(train_generator) / batch_size,
        validation_steps=len(valid_generator), callbacks=[reduce_lr])
    else:
        model = load_model('VGG16_CalTech101_best_model.h5')
        history = model.evaluate_generator(valid_generator)
        print(history)
    #print(model.metrics_names)
    model.save('VGG16_CalTech101_best_model.h5')
    save_model = ModelCheckpoint('VGG16_CalTech101_best_model_after_pruning.h5', monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
    return model, history


In [0]:


def my_get_all_dense_layers(model):
    all_dense_layers = list()
    for i,each_layer in enumerate(model.layers):
        if (each_layer.name[0:5] == 'dense'):
            all_dense_layers.append(i)
    return all_dense_layers[0:len(all_dense_layers)-1]

def my_get_all_conv_layers(model,first_time):
    all_conv_layers = list()
    for i,each_layer in enumerate(model.layers):
        # if (each_layer.name[0:6] == 'conv2d'):
        #     all_conv_layers.append(i)
        if 'conv' in each_layer.name:
            all_conv_layers.append(i)
    return all_conv_layers if (first_time==True) else all_conv_layers[1:]

def my_in_dense_layers_get_L1_norms_sorted_indices_and_values(model,graph):
    weights = list()
    all_dense_layers = my_get_all_dense_layers(model)
    for i in all_dense_layers:
        weights.append(model.layers[i].get_weights()[0])
    layer_wise_filter_sorted_indices_dense = list()
    layer_wise_filter_sorted_values_dense = list()
    for i in range(len(weights)):
        weight = weights[i]
        num_filters = len(weight[0,:])
        weights_dict = dict() 
        for j in range(num_filters):
            weights_sum = np.sum(abs(weight[:,j]))
            neuron = 'neuron {}'.format(j)
            weights_dict[neuron] = weights_sum
            # print(weights_dict[neuron])
        
        weights_dict_sorted = sorted(weights_dict.items(),key = lambda kv:kv[1]) 
        weight_values = list()
        neuron_indices = list()
        for element in weights_dict_sorted:
            neuron_indices.append(int(element[0][6:]))
            weight_values.append(element[1])
        layer_wise_filter_sorted_indices_dense.append(neuron_indices)
        layer_wise_filter_sorted_values_dense.append(weight_values)

        if graph == True:
                    x = np.arange(num_filters)
                    plt.figure(i+1,figsize=(7,5))
                    plt.plot(x,np.array(weight_values))
                    plt.axhline(y=np.mean(np.array(weight_values)),c='r')
                    for j in range(len(layer_wise_filter_sorted_values_dense[i])):
                        if(np.mean(np.array(weight_values)) < weight_values[j]):
                            plt.axvline(x=j,c='r')
                            break

                    plt.xlabel('filter number')
                    plt.ylabel('L1 norm')
                    plt.title('Conv2d_{}'.format(i+1))
                    plt.grid(True)
                    plt.style.use(['classic'])        

    return layer_wise_filter_sorted_indices_dense,layer_wise_filter_sorted_values_dense

def plot_histogram_l1_norms_dense(model):
    _,a = my_in_dense_layers_get_L1_norms_sorted_indices_and_values(my_model_vgg,False)
    all_dense_layers = my_get_all_dense_layers(model)
    for i in range(len(all_dense_layers)):     
        plt.figure(i,figsize=(7,7))
        plt.hist(a[i-1],density=1, bins=150) 
        plt.xlabel('L1 Norms of dense_{}'.format(i+1))
        plt.ylabel('Probability')
        plt.style.use(['classic'])

def my_in_conv_layers_get_L1_norms_sorted_indices_and_values(model,graph,first_time):
    weights = list()
    all_conv_layers = my_get_all_conv_layers(model,first_time)
    for i in all_conv_layers:
        # print('{}th layer'.format(i),all_conv_layers)
        weights.append(model.layers[i].get_weights()[0])
    layer_wise_filter_sorted_indices = list()
    layer_wise_filter_sorted_values = list()
    for i in range(len(weights)):
        weight = weights[i]
        num_filters = len(weight[0,0,0,:])
        # print(num_filters)
        weights_dict = dict() 
        for j in range(num_filters):
            weights_sum = np.sum(abs(weight[:,:,:,j]))
            filtr = 'filter {}'.format(j)
            weights_dict[filtr] = weights_sum
        
        weights_dict_sorted = sorted(weights_dict.items(),key = lambda kv:kv[1]) 
        # print('L1 norm of conv2D_{} layer'.format(i+1),weights_dict_sorted)

        weight_values = list()
        filter_indices = list()

        for element in weights_dict_sorted:
            filter_indices.append(int(element[0][6:]))  # extracting the index of filter from string (tentative try to come up with better code)
            weight_values.append(element[1]) 
    
        layer_wise_filter_sorted_indices.append(filter_indices)
        layer_wise_filter_sorted_values.append(weight_values)

        if graph == True:
                x = np.arange(num_filters)
                plt.figure(i+1,figsize=(7,7))
                plt.plot(x,np.array(weight_values))
                plt.axhline(y=np.mean(np.array(weight_values)),c='r')
                for j in range(len(layer_wise_filter_sorted_values[i-1])):
                    if(np.mean(np.array(weight_values)) < weight_values[j]):
                        plt.axvline(x=j,c='r')
                        break
                plt.style.use(['classic'])  


                plt.xlabel('filter number')
                plt.ylabel('L1 norm')
                plt.title('Conv2d_{}'.format(i+1))
                plt.grid(True)
                plt.style.use(['classic'])


    return layer_wise_filter_sorted_indices,layer_wise_filter_sorted_values

def plot_histogram_l1_norms_conv(model,first_time):
    _,a = my_in_conv_layers_get_L1_norms_sorted_indices_and_values(my_model_vgg,False,False)
    all_conv_layers = my_get_all_conv_layers(model,first_time)
    for i in range(len(all_conv_layers)):     
        plt.figure(i,figsize=(7,7))
        plt.hist(a[i-1],density=1, bins=150) 
        plt.xlabel('L1 Norms of conv_2d_{}'.format(i+1))
        plt.ylabel('Probability')
        plt.style.use(['classic'])

def my_get_prune_neuron_indices(model):
    layer_wise_neuron_sorted_indices,layer_wise_neuron_sorted_values = my_in_dense_layers_get_L1_norms_sorted_indices_and_values(model,False)
    all_dense_layers = my_get_all_dense_layers(model)
    thresh_hold_means = list()
    for i in range(len(all_dense_layers)):
        thresh_hold_means.append(np.mean(np.array(layer_wise_neuron_sorted_values[i])))
    prune_neuron_indices = list()
    for i in range(len(all_dense_layers)):
        #-----bsearch
        for j in range(len(layer_wise_neuron_sorted_values[i])):
            if(thresh_hold_means[i] < layer_wise_neuron_sorted_values[i][j]):
                prune_neuron_indices.append(j)
                break
    return prune_neuron_indices

def my_get_prune_filter_indices(model,first_time ):
    layer_wise_filter_sorted_indices,layer_wise_filter_sorted_values = my_in_conv_layers_get_L1_norms_sorted_indices_and_values(model, False,first_time)
    all_conv_layers = my_get_all_conv_layers(model,first_time)
    thresh_hold_means = list()
    for i in range(len(all_conv_layers)):
        # print(i,'/',len(all_conv_layers))
        thresh_hold_means.append(np.mean(np.array(layer_wise_filter_sorted_values[i])))
    prune_filter_indices = list()
    for i in range(len(all_conv_layers)):
        for j in range(len(layer_wise_filter_sorted_values[i])):
            if(thresh_hold_means[i] < layer_wise_filter_sorted_values[i][j]):
                prune_filter_indices.append(j)
                break
    return prune_filter_indices

def my_delete_filters(model,first_time):
    layer_wise_filter_sorted_indices,layer_wise_filter_sorted_values = my_in_conv_layers_get_L1_norms_sorted_indices_and_values(model,False,first_time)
    
    layer_wise_neuron_sorted_indices,layer_wise_neuron_sorted_values_ = my_in_dense_layers_get_L1_norms_sorted_indices_and_values(model,False)

    prune_filter_indices = my_get_prune_filter_indices(model,first_time)
    
    prune_neuron_indices = my_get_prune_neuron_indices(model)

    all_conv_layers = my_get_all_conv_layers(model,first_time)

    all_dense_layers = my_get_all_dense_layers(model)
    # print(all_conv_layers)
    surgeon = Surgeon(model)
    for index,value in enumerate(all_conv_layers):
        # print(value,index,prune_filter_indices[index])
        surgeon.add_job('delete_channels',model.layers[value],channels = layer_wise_filter_sorted_indices[index][0:prune_filter_indices[index]])
    
    for index,value in enumerate(all_dense_layers):
        # print(value,index,prune_filter_indices[index])
        surgeon.add_job('delete_channels',model.layers[value],channels = layer_wise_neuron_sorted_indices[index][0:prune_neuron_indices[index]])


    model_new = surgeon.operate()
    return model_new


# # This is temporary code cell you may delete it 
# my_new_model = my_vgg.model
# my_new_model = my_delete_filters(my_new_model,first_time=True)





In [4]:
model = VGG16(input_shape=(224, 224, 3), weights='imagenet', include_top=True)
#base_model.summary()
X = model.layers[-4].output
        # X = layers.MaxPooling2D(pool_size=(2, 2), strides=(1,1))(X)
        # X = layers.Flatten()(X)
        # # X = layers.Conv2D(512, kernel_size=(2, 2), strides=(1, 1))(X)
        # # X = layers.BatchNormalization()(X)
        # # X = layers.Activation('relu')(X)
        # # X = layers.Conv2D(256, kernel_size=(2, 2), strides=(1, 1))(X)
        # # X = layers.BatchNormalization()(X)
        # # X = layers.Activation('relu')(X)
        # # X = layers.GlobalMaxPooling2D()(X)
X = layers.Dense(512, activation='elu', kernel_initializer = 'he_normal')(X)
X = layers.BatchNormalization()(X)
X = layers.Dropout(0.4)(X)
X = layers.Dense(256, activation='selu', kernel_initializer = 'he_normal')(X)
X = layers.BatchNormalization()(X)
X = layers.Dropout(0.3)(X)

# uncomment the below line
# X = layers.Dense(101, activation='softmax', kernel_initializer='he_normal')(X)

# Delete the line below 
X = layers.Dense(2, activation='softmax', kernel_initializer='he_normal')(X)



model = models.Model(inputs=model.inputs, outputs=X)
#base_model.summary()
# for i in range(len(model.layers)-7):
#     model.layers[i].trainable = False
#base_model.summary()


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [5]:
my_get_all_conv_layers(model,True)

[1, 2, 4, 5, 7, 8, 9, 11, 12, 13, 15, 16, 17]

In [0]:
FirstTime = True
model, history = train(model, FirstTime)
# validation_accuracy = model.evaluate(x_test,y_test)[1]
# print(validation_accuracy)
# validation_accuracy = history.history[1]
validation_accuracy = max(history.history['val_acc'])
max_val_acc = validation_accuracy
count = 0
all_models = list()
while max_val_acc <= validation_accuracy   :
    FirstTime = False
    print("ITERATION {} ".format(count+1))
    all_models.append(model)
    if max_val_acc < validation_accuracy:
        max_val_acc = validation_accuracy

    if count < 1:
        model = my_delete_filters(model,True)

    else:
        model = my_delete_filters(model,False)
        
    model,history = train(model, True)
    validation_accuracy = max(history.history['val_acc'])
    model,history = train(model, True)
    validation_accuracy = max(history.history['val_acc'])
    print("VALIDATION ACCURACY AFTER {} ITERATIONS = {}".format(count+1,validation_accuracy))
    count+=1

#model.summary()

Found 220 images belonging to 2 classes.
Found 217 images belonging to 2 classes.



True



Epoch 1/20
4/3 [==================================] - 6s 1s/step - loss: 4.1163 - acc: 0.5625 - val_loss: 1.1921e-07 - val_acc: 0.5069
Epoch 2/20
4/3 [==================================] - 1s 345ms/step - loss: 7.4892 - acc: 0.5312 - val_loss: 1.1921e-07 - val_acc: 0.5069
Epoch 3/20
4/3 [==================================] - 2s 388ms/step - loss: 10.5775 - acc: 0.3438 - val_loss: 1.1921e-07 - val_acc: 0.5069
Epoch 4/20
4/3 [==================================] - 1s 373ms/step - loss: 6.1847 - acc: 0.5938 - val_loss: 1.1921e-07 - val_acc: 0.5069
Epoch 5/20
4/3 [==================================] - 1s 357ms/step - loss: 9.7403 - acc: 0.3750 - val_loss: 1.1921e-07 - val_acc: 0.5069
Epoch 6/20
4/3 [==================================] - 1s 362ms/step - loss: 8.0052 - acc: 0.4688 - val_loss: 1.1921e-07 - val_acc: 0.5069
Epoch 7/20
4/3 [==================================] - 2s 483ms/step - loss: 8.0576 - acc: 0.4750 - val_loss: 1.1921e-07 - val_acc: 0.5069
Epoch 8/20
4/3 [====================

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

INFO:root:
Unfortunately, your original traceback can not be constructed.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-fdce1fc6a0de>", line 25, in <module>
    model,history = train(model, True)
  File "<ipython-input-2-467d5dc37225>", line 60, in train
    validation_steps=len(valid_generator), callbacks=[reduce_lr])
  File "/usr/local/lib/python3.6/dist-packages/keras/legacy/interfaces.py", line 91, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/keras/engine/training.py", line 1658, in fit_generator
    initial_epoch=initial_epoch)
  File "/usr/local/lib/python3.6/dist-packages/keras/engine/training_generator.py", line 221, in fit_generator
    callbacks._call_batch_hook('train', 'end', batch_index, batch_logs)
  File "/usr/local/lib/python3.6/dist-packages/keras/callbacks.py", line 85, in _call_batch_hook
    batch_hook(batch, logs)
  File "

KeyboardInterrupt: ignored

ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/zmq/eventloop/zmqstream.py", line 432, in _run_callback
    callback(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/tornado/stack_context.py", line 277, in null_wrapper
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelbase.py", line 283, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelbase.py", line 235, in dispatch_shell
    handler(stream, idents, msg)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelbase.py", line 421, in execute_request
    self._abort_queues()
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelbase.py", line 637, in _abort_queues
    self._abort_queue(stream)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelbase.py", line 643, in _abort_queue
    ide